In [7]:
from cleaning import data_new, X_train, X_test, y_train, y_test,X,y,np,pd

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from random_forest import RandomForest
# Define your training data (X_train) and target variable (y_train)
# Define your testing data (X_test) and target variable (y_test)
# Make sure these variables are defined before running this code

# Define the preprocessor
numeric_features = X_train.select_dtypes(include=np.number).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Define the Random Forest Classifier model
rf_model = RandomForest()

# Define the feature selection using the model
select_model = SelectFromModel(estimator=rf_model)

# Create a pipeline
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', select_model),
    ('model', rf_model)
])

# Check the format of the target variable
if not pd.api.types.is_categorical_dtype(y_train):
    # If the target variable is not categorical, convert it to categorical
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

# Train the model pipeline
model_pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = model_pipeline.predict(X_test)

# Evaluate model performance (example using accuracy)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of model:", accuracy)


TypeError: Cannot clone object '<random_forest.RandomForest object at 0x7fe7ac700d30>' (type <class 'random_forest.RandomForest'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [11]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class RandomClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, random_state=None):
        """Initialize the classifier.
        
        Args:
            random_state (int, optional): Random state for reproducibility. Defaults to None.
        """
        self.random_state = random_state
        self.classes_ = None
        
    def fit(self, X, y):
        """Fit the classifier (learn the classes).
        
        Args:
            X (array-like): Training data (features).
            y (array-like): Target labels (classes).
        
        Returns:
            self: The fitted model.
        """
        # Store the unique classes in the training data
        self.classes_ = np.unique(y)
        
        # Set random seed for reproducibility (if provided)
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        # Return self to indicate the fit is complete
        return self
        
    def predict(self, X):
        """Predict class labels randomly.
        
        Args:
            X (array-like): Test data (features).
        
        Returns:
            array: Predicted class labels.
        """
        # Predict random classes based on the learned classes
        random_indices = np.random.randint(len(self.classes_), size=len(X))
        random_predictions = self.classes_[random_indices]
        
        return random_predictions
    
    def get_params(self, deep=True):
        """Get parameters for the estimator.
        
        Args:
            deep (bool, optional): Whether to get parameters recursively. Defaults to True.
        
        Returns:
            dict: Dictionary of parameters.
        """
        return {"random_state": self.random_state}
    
    def set_params(self, **params):
        """Set parameters for the estimator.
        
        Args:
            **params: Named parameters to set.
        
        Returns:
            self: The updated model.
        """
        for param, value in params.items():
            setattr(self, param, value)
        
        return self


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Assuming X_train, y_train, X_test, and y_test are defined

# Define your feature selection model
random_classifier = RandomClassifier(random_state=42)
select_model = SelectFromModel(estimator=random_classifier)

# Define the preprocessor
numeric_features = X_train.select_dtypes(include=np.number).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Create a pipeline
# Remove the feature selection step
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', random_classifier)
])


# Train the model pipeline
model_pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = model_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of model:", accuracy)

Accuracy of model: 0.5702479338842975
